In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv
import os
import pandas as pd
import numpy as np
import random
import torch.nn.init as init
import pdb
from sklearn.metrics import mean_absolute_percentage_error
torch.autograd.set_detect_anomaly(True)

In [3]:
seed = 42

def file_name(file_dir,file_type='.csv'):#默认为文件夹下的所有文件
    lst = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if(file_type == ''):
                lst.append(file)
            else:
                if os.path.splitext(file)[1] == str(file_type):#获取指定类型的文件名
                    lst.append(file)
    return lst

def normalize0(inputs):
    normalized = []
    for eq in inputs:
        maks = np.max(np.abs(eq))
        if maks != 0:
            normalized.append(eq / maks)
        else:
            normalized.append(eq)
    return np.array(normalized)


def normalize1(inputs):
    normalized = []
    for eq in inputs:
        mean = np.mean(eq)
        std = np.std(eq)
        if std != 0:
            normalized.append((eq - mean) / std)
        else:
            normalized.append(eq)
    return np.array(normalized)


def normalize(inputs):
    normalized = []
    for eq in inputs:
        with np.errstate(invalid='ignore'):
            eps = 1e-10  # 可以根据需要调整epsilon的值

            eq_log = [np.log(x + eps) if i < 5 else x for i, x in enumerate(eq)]

            #eq_log = [np.log(x) if i < 5 else x for i, x in enumerate(eq)]
            eq_log1 = np.nan_to_num(eq_log).tolist()
            normalized.append(eq_log1)
    return np.array(normalized)


def k_fold_split(inputs, targets, K, seed=None):
    # 确保所有随机操作都使用相同的种子
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)

    ind = int(len(inputs) / K)
    inputsK = []
    targetsK = []

    for i in range(0, K - 1):
        inputsK.append(inputs[i * ind:(i + 1) * ind])
        targetsK.append(targets[i * ind:(i + 1) * ind])

    inputsK.append(inputs[(i + 1) * ind:])
    targetsK.append(targets[(i + 1) * ind:])

    return inputsK, targetsK


def merge_splits(inputs, targets, k, K):
    if k != 0:
        z = 0
        inputsTrain = inputs[z]
        targetsTrain = targets[z]
    else:
        z = 1
        inputsTrain = inputs[z]
        targetsTrain = targets[z]

    for i in range(z + 1, K):
        if i != k:
            inputsTrain = np.concatenate((inputsTrain, inputs[i]))
            targetsTrain = np.concatenate((targetsTrain, targets[i]))

    return inputsTrain, targetsTrain, inputs[k], targets[k]


def targets_to_list(targets):
    targetList = np.array(targets)

    return targetList


In [4]:
class GCN(torch.nn.Module):
    """Graph Convolutional Network"""
    def __init__(self, dim_in, dim_h, dim_out):#[7,16,1]
        super().__init__()
        reg_const=0.0001
        self.gcn1 = GCNConv(dim_in, dim_h,weight_decay=reg_const,node_dim=1)
        self.gcn2 = GCNConv(dim_h, dim_h,weight_decay=reg_const,node_dim=1)
        self.gcn3 = GCNConv(dim_h, dim_h,bias=True,weight_decay=reg_const,node_dim=1)
        self.linear2 = torch.nn.Linear(dim_h, 8).double() 
        self.linear3 = torch.nn.Linear(8, dim_out).double() 
        self.initialize_weights()

    def initialize_weights(self):
        for layer in [self.gcn1, self.gcn2, self.gcn3, self.linear2, self.linear3]:
            if isinstance(layer, torch.nn.Linear):
                init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    init.zeros_(layer.bias)
            elif isinstance(layer, GCNConv):
                # For GCNConv, handle weight and bias separately
                init.xavier_uniform_(layer.lin.weight)
                if layer.bias is not None:
                    init.zeros_(layer.bias)


    def forward(self, x, adj):
        x=x.double()
        edge_index = torch.stack([np.nonzero(t)for t in torch.unbind(adj,dim=0)],dim=0).permute(0,2,1)
        edge_index=edge_index[0][[1, 0], :]
        print(edge_index.shape)
        self.gcn1 = self.gcn1.to(torch.double)
        self.gcn2 = self.gcn2.to(torch.double)
        self.gcn3 = self.gcn3.to(torch.double)
        h = self.gcn1(x, edge_index)
        h = torch.nn.LeakyReLU()(h)
        h = self.gcn2(h, edge_index)
        h = torch.nn.LeakyReLU()(h)
        h = self.gcn3(h, edge_index)
        h = torch.nn.LeakyReLU()(h)
        last_column = h[:, -1]
        conv1_new = last_column.unsqueeze(-1)
        #conv1_new = torch.nn.Flatten(conv1_new)
        #conv1_new = F.dropout(conv1_new,p=0.3)
        conv1_new = F.dropout(conv1_new.view(conv1_new.size(0), -1), p=0.3, training=self.training) 
        conv1_new = self.linear2(conv1_new)
        y_hat = self.linear3(conv1_new)
        return y_hat


    def fit(self,train_loader, val_loader,num_epochs=5000, patience=100):
        #best_val_loss = float('inf')
        best_val_loss = torch.tensor(float('inf'), dtype=torch.double)
        patience_counter = 0
        optimizer = optim.Adam(self.parameters(), lr=0.0015,weight_decay=5e-4)
        criterion = nn.MSELoss()
        # 使用平滑的 L1 损失，也称为 Huber loss
        # criterion = nn.SmoothL1Loss()


        for epoch in range(num_epochs):
            self.train()
            train_loss = 0.0 
            for inputs, graph_input, targets in train_loader:
                optimizer.zero_grad()
                outputs = self(inputs, graph_input)
                loss = criterion(outputs, targets.unsqueeze(1))
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm = 1)
                optimizer.step()
            #     train_loss += loss.item()
            # avg_train_loss = train_loss / len(train_loader)
            

            self.eval()
            val_loss = 0.0
            with torch.no_grad():
                for val_inputs, val_graph_input, val_targets in val_loader:
                    val_outputs = self(val_inputs, val_graph_input)
                    val_loss = criterion(val_outputs, val_targets.unsqueeze(1))
            #         val_loss += val_loss.item()
            # avg_val_loss=val_loss / len(val_loader)
            if (epoch + 1) % 20 == 0:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')
                

            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    return
                
                
    def test(self,test_loader):
        self.eval()
        predictions = []
        for test_inputs, test_graph_input, _ in test_loader:
            batch_predictions = self(test_inputs, test_graph_input)
            predictions.append(batch_predictions)
        predictions = torch.cat(predictions)
        return predictions

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Assuming your PyTorch model is defined as before


if __name__ == "__main__":
    # stock_info = sys.argv[0]
    # lag_bin = int(sys.argv[1])
    # lag_day = int(sys.argv[2])
    # bin_num = int(sys.argv[3])
    # random_state_here = int(sys.argv[4])
    # test_set_size = float(sys.argv[5])
    lag_bin = 3
    lag_day = 3
    num_nodes = (int(lag_bin)+1)*(int(lag_day)+1)
    forecast_days = 15
    bin_num=24
    random_state_here = 88
    mape_list = []
    data_dir = './data/volume/0308/'
    files =file_name('./data/')
    stocks_info = sorted(list(set(s.split('_25')[0] for s in files)))
    print(stocks_info)
    for stock_info in stocks_info[0:2]:
        print(f'>>>>>>>>>>>>>>>>>>>>{stock_info}>>>>>>>>>>>>>>>>>>>>>>>')
        data_dir1 = f'{data_dir}{stock_info}_{lag_bin}_{lag_day}'
        test_set_size = bin_num*forecast_days
        K = 5
        inputs_data = np.load(f'{data_dir1}_inputs.npy', allow_pickle=True)
        inputs_data = [[[torch.tensor(x, dtype=torch.float64) for x in sublist] for sublist in list1] for list1 in inputs_data]
        array_data = np.array(inputs_data)
        inputs = np.reshape(array_data, (len(inputs_data), num_nodes,-1))
        targets = np.load(f'{data_dir1}_output.npy', allow_pickle=True).astype(np.float64)
        graph_input = np.load(f'{data_dir1}_graph_input.npy', allow_pickle=True).astype(np.float64)
        graph_input = np.array([graph_input] * inputs.shape[0])
        graph_features = np.load(f'{data_dir1}_graph_coords.npy', allow_pickle=True).astype(np.float64)
        graph_features = np.array([graph_features] * inputs.shape[0])

        trainInputs, testInputs, traingraphInput, testgraphInput, traingraphFeature, testgraphFeature, trainTargets, testTargets = train_test_split(inputs, graph_input, graph_features, targets, test_size=test_set_size, 
                                                     random_state=random_state_here)
        testInputs = normalize(testInputs)
        # testInputs = test_inputs
        inputsK, targetsK = k_fold_split(trainInputs, trainTargets, K)

        mape_list = []

        test_dataset = TensorDataset(torch.tensor(testInputs), torch.tensor(testgraphInput), torch.tensor(testTargets))
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
        K = 5  # Number of folds
        for k in range(K):
            torch.manual_seed(0)  # Set a random seed for reproducibility

            trainInputsAll, trainTargets, valInputsAll, valTargets = merge_splits(inputsK, targetsK, k, K)

            trainGraphInput = traingraphInput[0:trainInputsAll.shape[0], :]
            trainGraphFeatureInput = traingraphFeature[0:trainInputsAll.shape[0], :]

            valGraphInput = traingraphInput[0:valInputsAll.shape[0], :]
            valGraphFeatureInput = traingraphFeature[0:valInputsAll.shape[0], :]

            trainInputs = normalize(trainInputsAll[:, :])
            valInputs = normalize(valInputsAll[:, :])

            # Assuming trainInputs, trainGraphInput, trainGraphFeatureInput, trainTargets are PyTorch tensors
            train_dataset = TensorDataset(torch.tensor(trainInputs), torch.tensor(trainGraphInput),torch.tensor(trainTargets))
            val_dataset = TensorDataset(torch.tensor(valInputs), torch.tensor(valGraphInput),torch.tensor(valTargets))

            # train_dataset = TensorDataset(torch.tensor(trainInputs, dtype=torch.float32), torch.tensor(trainGraphInput, dtype=torch.float32), torch.tensor(trainGraphFeatureInput, dtype=torch.float32), torch.tensor(trainTargets, dtype=torch.float32))
            # val_dataset = TensorDataset(torch.tensor(valInputs, dtype=torch.float32), torch.tensor(valGraphInput, dtype=torch.float32), torch.tensor(valGraphFeatureInput, dtype=torch.float32), torch.tensor(valTargets, dtype=torch.float32))

            train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=50, shuffle=False)

        
            model = GCN(7,8,1)
            print()
            model.fit(train_loader, val_loader)
            predictions=model.test(test_loader)
            torch.save(model.state_dict(), f'models/{stock_info}_{lag_bin}_{lag_day}_gcn_model_iteration_{k}.pt')
    
            print()
            print('Fold number:', k)

            new_predictions = np.array([item.detach().numpy() for item in predictions]).flatten()
            MAPE = []

            MAPE.append(mean_absolute_percentage_error(testTargets[:], new_predictions[:]))
            print(MAPE)
            testTargets0 = list(testTargets)

            res = {
                'testTargets': testTargets0,
                'new_predictions': new_predictions
            }

            res_df = pd.DataFrame(res)
            res_df.to_csv(f'./result/{stock_info}_{lag_bin}_{lag_day}_res_test_MAPE{k}.csv', index=False)

            print('MAPE = ', np.array(MAPE).mean())
            MAPE_mean = np.array(MAPE).mean()
            mape_list.append(MAPE)

        print('-')
        print('mape score = ', mape_list)


['000046_XSHE', '000753_XSHE', '000951_XSHE', '000998_XSHE', '002282_XSHE', '002679_XSHE', '002841_XSHE', '002882_XSHE', '300133_XSHE', '300174_XSHE', '300263_XSHE', '300343_XSHE', '300433_XSHE', '300540_XSHE', '600622_XSHG', '603053_XSHG', '603095_XSHG', '603359_XSHG']
>>>>>>>>>>>>>>>>>>>>000046_XSHE>>>>>>>>>>>>>>>>>>>>>>>

torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])


/usr/local/anaconda3/envs/bioinfo/lib/python3.9/site-packages/torch/autograd/__init__.py:197: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /opt/conda/conda-bld/pytorch_1670525539683/work/c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])
torch.Size([2, 42])


KeyboardInterrupt: 

In [32]:
x=np.random.randint(0,10,size=50*16*16).reshape([50,16,16])

In [61]:
np.shape(torch.stack([np.nonzero(t)for t in torch.unbind( torch.randn(10,20,30),dim=0)],dim=0))

torch.Size([10, 600, 2])

In [83]:
import torch

def int_to_one_hot(integer, num_classes):
    one_hot = torch.nn.functional.one_hot(torch.tensor(integer - 1), num_classes=num_classes)
    return one_hot

# 例子：将整数编码 5 转换为 one-hot 编码
integer_code = 5
num_classes = 24  # 一天被划分为 24 个小时
time_one_hot = int_to_one_hot(integer_code, num_classes)

print(time_one_hot.numpy())


[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [261]:
import numpy as np

def one_hot_encoding(i, total_classes):
    one_hot_vector = np.zeros(total_classes)
    one_hot_vector[i] = 1
    return one_hot_vector

# 示例：生成长度为10的One-Hot向量，其中索引为3的位置是1，其他位置是0
index = 3
total_classes = 16
result = one_hot_encoding(index, total_classes)

print(result)


[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [262]:
import numpy as np

# 生成随机16x16矩阵
A = np.random.rand(16, 16)

print(A)


[[0.74049304 0.2683107  0.64684136 0.48311667 0.77103438 0.65039313
  0.72921997 0.32415868 0.06038467 0.90408672 0.72370617 0.53347065
  0.42896676 0.66994356 0.83883116 0.50870367]
 [0.00256826 0.07413307 0.94638582 0.86008728 0.98315507 0.90569552
  0.81407539 0.28489578 0.09129948 0.7321216  0.53827818 0.50693083
  0.82056561 0.54649232 0.7315669  0.38931277]
 [0.28372526 0.99687045 0.55981915 0.78662249 0.32212174 0.74443956
  0.15220498 0.28171196 0.70329487 0.5579942  0.70249792 0.57378471
  0.11665191 0.91947194 0.51315786 0.10236673]
 [0.63889845 0.44305818 0.53023973 0.82083926 0.24213653 0.13071741
  0.94463156 0.14474002 0.61855058 0.34781547 0.78347262 0.32049134
  0.14708563 0.66103614 0.16192767 0.14915083]
 [0.05886012 0.96334302 0.05970044 0.29037624 0.13125891 0.01666517
  0.8223141  0.81990527 0.17862031 0.24181944 0.06319064 0.12500831
  0.84252535 0.45658616 0.55880523 0.55723651]
 [0.73654614 0.62155648 0.19919113 0.94459622 0.63661815 0.11880613
  0.16613741 0.55

In [270]:
e_concat_list=[]
for i in range(16):
    a=A[:,i]
    print(a)
    e_concat_list.append(a)
    

[0.74049304 0.00256826 0.28372526 0.63889845 0.05886012 0.73654614
 0.28708771 0.83546937 0.57834996 0.17227078 0.66026991 0.22797302
 0.70927426 0.72757187 0.28182515 0.06106333]
[0.2683107  0.07413307 0.99687045 0.44305818 0.96334302 0.62155648
 0.01078129 0.28922117 0.46861726 0.98984268 0.38462816 0.15072159
 0.144742   0.27461974 0.97685136 0.40563647]
[0.64684136 0.94638582 0.55981915 0.53023973 0.05970044 0.19919113
 0.62070058 0.88789772 0.98555572 0.03599855 0.04693087 0.04085763
 0.51682168 0.53040689 0.22257515 0.3658274 ]
[0.48311667 0.86008728 0.78662249 0.82083926 0.29037624 0.94459622
 0.82117462 0.44065017 0.16317134 0.81575405 0.75977408 0.24887479
 0.67702802 0.42948145 0.39779537 0.44601216]
[0.77103438 0.98315507 0.32212174 0.24213653 0.13125891 0.63661815
 0.53703659 0.40229796 0.8365532  0.26675859 0.27126093 0.83138612
 0.36035871 0.12590623 0.44484208 0.54943237]
[0.65039313 0.90569552 0.74443956 0.13071741 0.01666517 0.11880613
 0.62766885 0.1383436  0.31640019

In [352]:
def _prepare_attentional_mechanism_input(Wh):
    # Wh.shape (N, out_feature)
    # self.a.shape (2 * out_feature, 1)
    # Wh1&2.shape (N, 1)
    # e.shape (N, N)
    E=[]
    for i in range(16):
        a=A[:,i].unsqueeze(1)
        Wh1 = torch.matmul(Wh, a[:8, :])
        Wh2 = torch.matmul(Wh, a[8:, :])
        # broadcast add
        e= Wh1 + Wh2.T
        E.append(e[i,:])
        E_matrix = torch.stack(E, dim=0)
    return E_matrix

In [353]:
A = torch.tensor(np.random.rand(16, 16))
B = torch.tensor(np.random.rand(16, 8))

In [355]:
e=_prepare_attentional_mechanism_input(B)
print(e.shape)

torch.Size([16, 16])


In [291]:
A = nn.Parameter(torch.empty(size=(2*8, 1)))

In [292]:
A

Parameter containing:
tensor([[ 5.5813e-11],
        [ 3.0628e-41],
        [-8.8168e+12],
        [ 6.5540e-01],
        [ 1.6507e-14],
        [ 1.2486e+00],
        [ 1.0065e+27],
        [ 1.1304e+00],
        [ 6.1175e-01],
        [-1.3547e+00],
        [-3.4584e+24],
        [ 8.7174e-01],
        [-1.8568e-02],
        [ 1.3618e+00],
        [ 2.0452e-05],
        [ 1.4018e+00]], requires_grad=True)

In [356]:
print(torch.cuda.is_available())

True
